### Breast cancer analysis
Using the dataset we try to find feaures to help determine which features play an important role in differentiating between a malignant and benign tumor by:
-  Data exploration and analysis
-  Feature selection using data visualisation
-  train machine learning model onto dataset to help make predictions for future use

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
%pylab inline
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import KFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn import metrics

Populating the interactive namespace from numpy and matplotlib


C:\Users\aashe\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
df = pd.read_csv('data.csv', header = 0)
df.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,NaN


Removing features that would not be useful for our prediction

In [3]:
df.drop('id',axis=1,inplace=True)
df.drop('Unnamed: 32',axis=1,inplace=True)

converting diagnosis to bianary variables, 1 = Malignant, 0 = Benign

In [4]:
df['diagnosis'] = df['diagnosis'].map({'M':1, 'B':0})
df.head()

,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,1,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,1,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,1,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,1,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [5]:
df.describe()

,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
count,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,...,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000
mean,0.372583,14.127292,19.289649,91.969033,654.889104,0.096360,0.104341,0.088799,0.048919,0.181162,...,16.269190,25.677223,107.261213,880.583128,0.132369,0.254265,0.272188,0.114606,0.290076,0.083946
std,0.483918,3.524049,4.301036,24.298981,351.914129,0.014064,0.052813,0.079720,0.038803,0.027414,...,4.833242,6.146258,33.602542,569.356993,0.022832,0.157336,0.208624,0.065732,0.061867,0.018061
min,0.000000,6.981000,9.710000,43.790000,143.500000,0.052630,0.019380,0.000000,0.000000,0.106000,...,7.930000,12.020000,50.410000,185.200000,0.071170,0.027290,0.000000,0.000000,0.156500,0.055040
25%,0.000000,11.700000,16.170000,75.170000,420.300000,0.086370,0.064920,0.029560,0.020310,0.161900,...,13.010000,21.080000,84.110000,515.300000,0.116600,0.147200,0.114500,0.064930,0.250400,0.071460
50%,0.000000,13.370000,18.840000,86.240000,551.100000,0.095870,0.092630,0.061540,0.033500,0.179200,...,14.970000,25.410000,97.660000,686.500000,0.131300,0.211900,0.226700,0.099930,0.282200,0.080040
75%,1.000000,15.780000,21.800000,104.100000,782.700000,0.105300,0.130400,0.130700,0.074000,0.195700,...,18.790000,29.720000,125.400000,1084.000000,0.146000,0.339100,0.382900,0.161400,0.317900,0.092080
max,1.000000,28.110000,39.280000,188.500000,2501.000000,0.163400,0.345400,0.426800,0.201200,0.304000,...,36.040000,49.540000,251.200000,4254.000000,0.222600,1.058000,1.252000,0.291000,0.663800,0.207500


In [6]:
df.diagnosis.value_counts()

0    357
1    212
Name: diagnosis, dtype: int64

## Data visualisation

In [7]:
feature_mean = list(df.columns[1:11])
dfm = df[df['diagnosis']==1]
dfb = df[df['diagnosis']==0]

In [8]:
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.figure_factory as ff
import plotly.tools as tls

We try to see which features show a distinction between malignant and benign tumors using the help of data visualisation

In [9]:
radius_mean1 = df[df['diagnosis']==1]['radius_mean']
radius_mean0 = df[df['diagnosis']==0]['radius_mean']

trace_rm1 = go.Histogram(x = radius_mean1, opacity = 0.75, name = 'malignant')
trace_rm2 = go.Histogram(x = radius_mean0, opacity = 0.75, name = 'benign')

data1 = [trace_rm1,trace_rm2]
layout1 = go.Layout(barmode = 'overlay', title = 'radius mean')
fig1 = go.Figure(data=data1, layout=layout1)

py.iplot(fig1)

In [10]:
texture_mean1 = df[df['diagnosis']==1]['texture_mean']
texture_mean0 = df[df['diagnosis']==0]['texture_mean']

trace_tm1 = go.Histogram(x = texture_mean1, opacity = 0.75, name = 'malignant')
trace_tm2 = go.Histogram(x = texture_mean0, opacity = 0.75, name = 'benign')

data2 = [trace_tm1,trace_tm2]
layout2 = go.Layout(barmode = 'overlay', title = 'texture mean')
fig2  = go.Figure(data=data2, layout=layout2)

py.iplot(fig2)

In [11]:
trace_5 = go.Histogram(x = df[df['diagnosis']==1]['perimeter_mean'], opacity = 0.75, )
trace_6 = go.Histogram(x = df[df['diagnosis']==0]['perimeter_mean'], opacity = 0.75, )
trace_7 = go.Histogram(x = df[df['diagnosis']==1]['area_mean'], opacity = 0.75, )
trace_8 = go.Histogram(x = df[df['diagnosis']==0]['area_mean'], opacity = 0.75, )
trace_9 = go.Histogram(x = df[df['diagnosis']==1]['smoothness_mean'], opacity = 0.75, )
trace_10 = go.Histogram(x = df[df['diagnosis']==0]['smoothness_mean'], opacity = 0.75, )
trace_11 = go.Histogram(x = df[df['diagnosis']==1]['compactness_mean'], opacity = 0.75, )
trace_12 = go.Histogram(x = df[df['diagnosis']==0]['compactness_mean'], opacity = 0.75, )
trace_13 = go.Histogram(x = df[df['diagnosis']==1]['concavity_mean'], opacity = 0.75, )
trace_14 = go.Histogram(x = df[df['diagnosis']==0]['concavity_mean'], opacity = 0.75, )
trace_15 = go.Histogram(x = df[df['diagnosis']==1]['concave points_mean'], opacity = 0.75, )
trace_16 = go.Histogram(x = df[df['diagnosis']==0]['concave points_mean'], opacity = 0.75, )
trace_17 = go.Histogram(x = df[df['diagnosis']==1]['symmetry_mean'], opacity = 0.75, )
trace_18 = go.Histogram(x = df[df['diagnosis']==0]['symmetry_mean'], opacity = 0.75, )
trace_19 = go.Histogram(x = df[df['diagnosis']==1]['fractal_dimension_mean'], opacity = 0.75, )
trace_20 = go.Histogram(x = df[df['diagnosis']==0]['fractal_dimension_mean'], opacity = 0.75, )

data3 = [trace_5,trace_6]
data4 = [trace_7,trace_8]
data5 = [trace_9,trace_10]
data6 = [trace_11,trace_14]
data7 = [trace_13,trace_14]
data8 = [trace_15,trace_16]
data9 = [trace_17,trace_18]
data10 = [trace_19,trace_20]

fig3  = go.Figure(data=data3, layout=layout1)
fig4  = go.Figure(data=data4, layout=layout1)
fig5  = go.Figure(data=data5, layout=layout1)
fig6  = go.Figure(data=data6, layout=layout1)
fig7  = go.Figure(data=data7, layout=layout1)
fig8  = go.Figure(data=data8, layout=layout1)
fig9  = go.Figure(data=data9, layout=layout1)
fig10  = go.Figure(data=data10, layout=layout1)

In [12]:
fig = tls.make_subplots(rows =5, cols =2, subplot_titles =('radius mean', 'texture mean','perimeter mean', 'area mean','smootheness mean', 'compactness mean','concavity mean', 'concave points mean', 'symmetry mean', 'fractal dimension mean'))
fig.append_trace(fig1['data'][1], 1, 1)
fig.append_trace(fig1['data'][0], 1, 1)
fig.append_trace(fig2['data'][1], 1, 2)
fig.append_trace(fig2['data'][0], 1, 2)
fig.append_trace(fig3['data'][1], 2, 1)
fig.append_trace(fig3['data'][0], 2, 1)
fig.append_trace(fig4['data'][1], 2, 2)
fig.append_trace(fig4['data'][0], 2, 2)
fig.append_trace(fig5['data'][1], 3, 1)
fig.append_trace(fig5['data'][0], 3, 1)
fig.append_trace(fig6['data'][1], 3, 2)
fig.append_trace(fig6['data'][0], 3, 2)
fig.append_trace(fig7['data'][1], 4, 1)
fig.append_trace(fig7['data'][0], 4, 1)
fig.append_trace(fig8['data'][1], 4, 2)
fig.append_trace(fig8['data'][0], 4, 2)
fig.append_trace(fig9['data'][1], 5, 1)
fig.append_trace(fig9['data'][0], 5, 1)
fig.append_trace(fig10['data'][1], 5, 2)
fig.append_trace(fig10['data'][0], 5, 2)
fig['layout'].update(showlegend= False)
py.iplot(fig)

This is the format of your plot grid:
[ (1,1) x1,y1 ]    [ (1,2) x2,y2 ]  
[ (2,1) x3,y3 ]    [ (2,2) x4,y4 ]  
[ (3,1) x5,y5 ]    [ (3,2) x6,y6 ]  
[ (4,1) x7,y7 ]    [ (4,2) x8,y8 ]  
[ (5,1) x9,y9 ]    [ (5,2) x10,y10 ]



We can see that the following features can show a distinction bettern malignant and benign tumors :

    Radius mean, perimeter mean, area mean, compactness mean, concavity mean, concave points mean
    
So we drop the other features before runnig our machine learning model

In [13]:
dfx = df.copy()
dfx.drop(['texture_mean', 'smoothness_mean', 'symmetry_mean','fractal_dimension_mean'], axis =1 , inplace = True)
dfx.head()

,diagnosis,radius_mean,perimeter_mean,area_mean,compactness_mean,concavity_mean,concave points_mean,radius_se,texture_se,perimeter_se,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,1,17.99,122.80,1001.0,0.27760,0.3001,0.14710,1.0950,0.9053,8.589,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,1,20.57,132.90,1326.0,0.07864,0.0869,0.07017,0.5435,0.7339,3.398,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,1,19.69,130.00,1203.0,0.15990,0.1974,0.12790,0.7456,0.7869,4.585,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,1,11.42,77.58,386.1,0.28390,0.2414,0.10520,0.4956,1.1560,3.445,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,1,20.29,135.10,1297.0,0.13280,0.1980,0.10430,0.7572,0.7813,5.438,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [14]:
dfy = dfx.diagnosis
dfx.drop('diagnosis', axis = 1 , inplace = True)

In [15]:
X_train, X_test, y_train, y_test = train_test_split(dfx, dfy, test_size = 0.2, random_state = 10)

In [16]:
rf = RandomForestClassifier(n_estimators = 100).fit(X_train, y_train )
rf.score(X_test, y_test)

0.98245614035087714